# Other get options

#### Sections
* [Post-processing options](#Post-processing-options)
* [`tzout`](#tzout)
* [`fill`](#fill)
* [`include` and `exclude`](#include-and-exclude)
* [`side`](#side)
* [`close_only`](#close_only)
* [`lose_single_symbol`](#lose_single_symbol)

#### Note

The cell **outputs** shown in this tutorial are based on executing the cells at the time shown in the output of the following cell. Simply rerun the cells to bring any dynamic output up to date.

In [1]:
import pandas as pd
from zoneinfo import ZoneInfo
now = pd.Timestamp.now(tz=ZoneInfo("UTC")).floor("T")
print(f"{now!r}")
print(f"{now.astimezone(ZoneInfo('America/New_York'))!r}")

Timestamp('2023-09-09 19:03:00+0000', tz='UTC')
Timestamp('2023-09-09 15:03:00-0400', tz='America/New_York')


## Setup

Run the following cell to import tutorial dependencies.

In [2]:
from market_prices import PricesYahoo
from market_prices.support import tutorial_helpers as th
from market_prices import helpers

Run the following cell to instantiate prices objects and define values used in this tutorial.

In [3]:
_prices = PricesYahoo("MSFT, 9988.HK, AZN.L", lead_symbol="MSFT")
xnys = _prices.calendar_default
xhkg = _prices.calendars["9988.HK"]
_sessions_range = th.get_sessions_range_for_bi(_prices, _prices.bis.T1)
length = pd.Timedelta(hours=18, minutes=30)
session = th.get_conforming_cc_sessions(_prices.cc, length, *_sessions_range, 1)[0]
start_H1 = th.get_sessions_range_for_bi(_prices, _prices.bis.H1)[0]
start_H1_oob = helpers.to_tz_naive(xnys.session_offset(start_H1, -2))

## Post-processing options

This tutorial covers `get` options available to **format and tidy** the price table.

The `get` method implements these options by calling the price table's `.pt.operate` method (see the [Tidying data](./pt_accessor.ipynb#Tidying-data) section of the [pt accessor](./pt_accessor.ipynb) tutorial). Rather than passing these options to `get`, the same effect could be achieved by subsequently passing the options to the `.pt.operate` method. The options are included to `get` simply as a convenience.

Be clear, these options are implemented as **post-processing operations** undertaken to the price table. They do not have any effect on the table interval, the period represented by each indice or the overall period covered by the price table.

The following tables include none of these options and will be used for the purpose of comparison.

In [4]:
prices = PricesYahoo("MSFT, 9988.HK, AZN.L", lead_symbol="MSFT")
session # for reference

Timestamp('2023-08-11 00:00:00')

In [5]:
kwargs_daily = dict(interval="1D", end=session, days=10)
df_daily = prices.get(**kwargs_daily)
df_daily

symbol            MSFT                                                \
                  open        high         low       close    volume   
2023-07-31  336.920013  337.700012  333.359985  335.920013  25446000   
2023-08-01  335.190002  338.540009  333.700012  336.339996  18311900   
2023-08-02  333.630005  333.630005  326.359985  327.500000  27761300   
2023-08-03  326.000000  329.880005  325.950012  326.660004  18253700   
2023-08-04  331.880005  335.140015  327.239990  327.779999  23727700   
2023-08-07  328.369995  331.109985  327.519989  330.109985  17741500   
2023-08-08  326.959991  328.750000  323.000000  326.049988  22327600   
2023-08-09  326.470001  327.109985  321.049988  322.230011  22373300   
2023-08-10  326.019989  328.260010  321.179993  322.929993  20113700   
2023-08-11  320.260010  322.410004  319.209991  321.010010  24342600   

symbol        9988.HK                                                AZN.L  \
                 open        high        low      close    volume     open   
2023-07-31  99.000000   99.750000  96.800003  97.500000  74290881  11028.0   
2023-08-01  99.699997  100.000000  96.900002  97.849998  53949339  11176.0   
2023-08-02  97.650002   98.599998  94.599998  95.150002  37885551  11040.0   
2023-08-03  93.699997   94.750000  93.050003  93.150002  50021614  11018.0   
2023-08-04  95.250000   96.599998  94.400002  95.150002  32949492  10952.0   
2023-08-07  94.349998   96.050003  93.599998  95.599998  22359341  10894.0   
2023-08-08  94.099998   94.300003  92.699997  93.050003  35528837  10940.0   
2023-08-09  92.349998   94.400002  92.050003  94.099998  30298227  11110.0   
2023-08-10  93.800003   94.699997  92.349998  94.300003  34598900  11166.0   
2023-08-11  97.150002   97.949997  95.300003  95.300003  61924422  11066.0   

symbol                                               
               high           low    close   volume  
2023-07-31  11204.0  11022.000000  11186.0  1639504  
2023-08-01  11254.0  11148.000000  11196.0   955804  
2023-08-02  11140.0  10874.000000  11096.0  1371970  
2023-08-03  11018.0  10778.000000  10900.0  4910009  
2023-08-04  10968.0  10784.000000  10910.0  1506764  
2023-08-07  10944.0  10804.000000  10886.0   853789  
2023-08-08  11100.0  10834.000000  11030.0  1977417  
2023-08-09  11212.0  10980.642578  11182.0  2545313  
2023-08-10  11184.0  11042.000000  11162.0  2848203  
2023-08-11  11124.0  10912.000000  11030.0  2243755

In [6]:
kwargs_intraday = dict(end=session, days=2)
df_intraday = prices.get(**kwargs_intraday)
df_intraday

symbol                                            MSFT              \
                                                  open        high   
[2023-08-10 09:30:00, 2023-08-10 09:35:00)  326.015015  326.890015   
[2023-08-10 09:35:00, 2023-08-10 09:40:00)  326.149994  326.636505   
[2023-08-10 09:40:00, 2023-08-10 09:45:00)  326.200012  326.570007   
[2023-08-10 09:45:00, 2023-08-10 09:50:00)  325.450012  326.510010   
[2023-08-10 09:50:00, 2023-08-10 09:55:00)  326.399994  326.579987   
...                                                ...         ...   
[2023-08-11 15:35:00, 2023-08-11 15:40:00)  321.720001  321.829987   
[2023-08-11 15:40:00, 2023-08-11 15:45:00)  321.339996  321.429993   
[2023-08-11 15:45:00, 2023-08-11 15:50:00)  321.350006  321.399994   
[2023-08-11 15:50:00, 2023-08-11 15:55:00)  321.190002  321.440002   
[2023-08-11 15:55:00, 2023-08-11 16:00:00)  320.894989  321.269989   

symbol                                                                         \
                                                   low       close     volume   
[2023-08-10 09:30:00, 2023-08-10 09:35:00)  325.500000  326.189911  1094887.0   
[2023-08-10 09:35:00, 2023-08-10 09:40:00)  325.489990  326.160004   365134.0   
[2023-08-10 09:40:00, 2023-08-10 09:45:00)  325.109985  325.340088   345965.0   
[2023-08-10 09:45:00, 2023-08-10 09:50:00)  325.450012  326.390015   304905.0   
[2023-08-10 09:50:00, 2023-08-10 09:55:00)  325.950012  326.440002   325788.0   
...                                                ...         ...        ...   
[2023-08-11 15:35:00, 2023-08-11 15:40:00)  321.279999  321.353210   165967.0   
[2023-08-11 15:40:00, 2023-08-11 15:45:00)  321.179993  321.338013   217661.0   
[2023-08-11 15:45:00, 2023-08-11 15:50:00)  320.910004  321.179993   241240.0   
[2023-08-11 15:50:00, 2023-08-11 15:55:00)  320.859985  320.910004   389375.0   
[2023-08-11 15:55:00, 2023-08-11 16:00:00)  320.589996  321.089996  1342284.0   

symbol                                     9988.HK                        \
                                              open high low close volume   
[2023-08-10 09:30:00, 2023-08-10 09:35:00)     NaN  NaN NaN   NaN    NaN   
[2023-08-10 09:35:00, 2023-08-10 09:40:00)     NaN  NaN NaN   NaN    NaN   
[2023-08-10 09:40:00, 2023-08-10 09:45:00)     NaN  NaN NaN   NaN    NaN   
[2023-08-10 09:45:00, 2023-08-10 09:50:00)     NaN  NaN NaN   NaN    NaN   
[2023-08-10 09:50:00, 2023-08-10 09:55:00)     NaN  NaN NaN   NaN    NaN   
...                                            ...  ...  ..   ...    ...   
[2023-08-11 15:35:00, 2023-08-11 15:40:00)     NaN  NaN NaN   NaN    NaN   
[2023-08-11 15:40:00, 2023-08-11 15:45:00)     NaN  NaN NaN   NaN    NaN   
[2023-08-11 15:45:00, 2023-08-11 15:50:00)     NaN  NaN NaN   NaN    NaN   
[2023-08-11 15:50:00, 2023-08-11 15:55:00)     NaN  NaN NaN   NaN    NaN   
[2023-08-11 15:55:00, 2023-08-11 16:00:00)     NaN  NaN NaN   NaN    NaN   

symbol                                        AZN.L                    \
                                               open     high      low   
[2023-08-10 09:30:00, 2023-08-10 09:35:00)  11118.0  11132.0  11114.0   
[2023-08-10 09:35:00, 2023-08-10 09:40:00)  11122.0  11134.0  11112.0   
[2023-08-10 09:40:00, 2023-08-10 09:45:00)  11114.0  11138.0  11114.0   
[2023-08-10 09:45:00, 2023-08-10 09:50:00)  11124.0  11138.0  11118.0   
[2023-08-10 09:50:00, 2023-08-10 09:55:00)  11120.0  11140.0  11116.0   
...                                             ...      ...      ...   
[2023-08-11 15:35:00, 2023-08-11 15:40:00)      NaN      NaN      NaN   
[2023-08-11 15:40:00, 2023-08-11 15:45:00)      NaN      NaN      NaN   
[2023-08-11 15:45:00, 2023-08-11 15:50:00)      NaN      NaN      NaN   
[2023-08-11 15:50:00, 2023-08-11 15:55:00)      NaN      NaN      NaN   
[2023-08-11 15:55:00, 2023-08-11 16:00:00)      NaN      NaN      NaN   

symbol                                                        
                                 

### `tzout`

The `tzout` parameter provides for setting the index timezone.

Indexes comprising dates, for example for daily or monthly data, are by default timezone-naive.

In [7]:
df_daily.pt.tz == None

True

It makes little sense for a date to have a timezone, although tzout does provide for setting dates to "UTC" should that be required.

In [8]:
df = prices.get(**kwargs_daily, tzout="UTC")
df[:1]

symbol                           MSFT                                      \
                                 open        high         low       close   
2023-07-31 00:00:00+00:00  336.920013  337.700012  333.359985  335.920013   

symbol                              9988.HK                                    \
                             volume    open   high        low close    volume   
2023-07-31 00:00:00+00:00  25446000    99.0  99.75  96.800003  97.5  74290881   

symbol                       AZN.L                                      
                              open     high      low    close   volume  
2023-07-31 00:00:00+00:00  11028.0  11204.0  11022.0  11186.0  1639504

In [9]:
df.pt.tz

zoneinfo.ZoneInfo(key='UTC')

By default the timezone of intraday prices is set to the same timezone that the `tzin` parameter evaluates to (whether passed or not). If neither `tzin` or `lead_symbol` are passed then this will be the default timezone of the prices instance.

In [10]:
prices.tz_default == df_intraday.pt.tz

True

If `lead_symbol` is passed then `tzin` and hence `tzout` will default to the timezone associated with the lead symbol.

In [11]:
df = prices.get(**kwargs_intraday, lead_symbol="9988.HK")
df[:1]

symbol                                     MSFT                       9988.HK  \
                                           open high low close volume    open   
[2023-08-10 09:30:00, 2023-08-10 09:35:00)  NaN  NaN NaN   NaN    NaN   93.75   

symbol                                                                         \
                                                 high        low close volume   
[2023-08-10 09:30:00, 2023-08-10 09:35:00)  93.800003  93.300003  93.5    0.0   

symbol                                     AZN.L                        
                                            open high low close volume  
[2023-08-10 09:30:00, 2023-08-10 09:35:00)   NaN  NaN NaN   NaN    NaN

In [12]:
df.pt.tz == prices.timezones["9988.HK"]

True

And if `tzin` is passed then that takes precedence and `tzout` defaults to the same timezone as `tzin`. In the following example `end` is passed as a tz-naive timestamp and `tzin` is passed to define the timezone that should be attributed to `end` as London.

In [13]:
xlon = prices.calendars["AZN.L"]
tz = prices.timezones["AZN.L"]
end = xlon.session_close(session).astimezone(tz).astimezone(None)
kwargs = {**kwargs_intraday, **{"end":end}}
print(kwargs, "\n", tz)  # for reference

{'end': Timestamp('2023-08-11 15:30:00'), 'days': 2} 
 Europe/London


In [14]:
df = prices.get(**kwargs, tzin=tz)
df[:1]

symbol                                            MSFT              \
                                                  open        high   
[2023-08-09 15:30:00, 2023-08-09 15:35:00)  324.200012  324.579987   

symbol                                                                        \
                                                   low       close    volume   
[2023-08-09 15:30:00, 2023-08-09 15:35:00)  324.190002  324.339996  161779.0   

symbol                                     9988.HK                        \
                                              open high low close volume   
[2023-08-09 15:30:00, 2023-08-09 15:35:00)     NaN  NaN NaN   NaN    NaN   

symbol                                        AZN.L                    \
                                               open     high      low   
[2023-08-09 15:30:00, 2023-08-09 15:35:00)  11184.0  11190.0  11178.0   

symbol                                                       
                                              close  volume  
[2023-08-09 15:30:00, 2023-08-09 15:35:00)  11188.0  8633.0

In [15]:
df.pt.tz == tz

True

To define a non-default output timezone, simply pass `tzout`. `tzout` can be defined in the same ways as `tzin`:
* a string that's valid input to `zoneinfo.ZoneInfo`.
* an instance of `zoneinfo.Zoneinfo`.
* a string of a symbol.

Note that in each of the following examples the price data is the same, although the index timezone may change.

In [16]:
df = prices.get(**kwargs_intraday, tzout="UTC")
df[:1]

symbol                                            MSFT                     \
                                                  open        high    low   
[2023-08-10 13:30:00, 2023-08-10 13:35:00)  326.015015  326.890015  325.5   

symbol                                                            9988.HK  \
                                                 close     volume    open   
[2023-08-10 13:30:00, 2023-08-10 13:35:00)  326.189911  1094887.0     NaN   

symbol                                                              AZN.L  \
                                           high low close volume     open   
[2023-08-10 13:30:00, 2023-08-10 13:35:00)  NaN NaN   NaN    NaN  11118.0   

symbol                                                                         
                                               high      low    close  volume  
[2023-08-10 13:30:00, 2023-08-10 13:35:00)  11132.0  11114.0  11120.0  8815.0

In [17]:
df.pt.tz

zoneinfo.ZoneInfo(key='UTC')

In [18]:
tz = prices.timezones["9988.HK"]
print(f"{tz=}\n")  # for reference
df = prices.get(**kwargs_intraday, tzout=tz)
df[:1]

tz=zoneinfo.ZoneInfo(key='Asia/Hong_Kong')



symbol                                            MSFT                     \
                                                  open        high    low   
[2023-08-10 21:30:00, 2023-08-10 21:35:00)  326.015015  326.890015  325.5   

symbol                                                            9988.HK  \
                                                 close     volume    open   
[2023-08-10 21:30:00, 2023-08-10 21:35:00)  326.189911  1094887.0     NaN   

symbol                                                              AZN.L  \
                                           high low close volume     open   
[2023-08-10 21:30:00, 2023-08-10 21:35:00)  NaN NaN   NaN    NaN  11118.0   

symbol                                                                         
                                               high      low    close  volume  
[2023-08-10 21:30:00, 2023-08-10 21:35:00)  11132.0  11114.0  11120.0  8815.0

In [19]:
df.pt.tz

zoneinfo.ZoneInfo(key='Asia/Hong_Kong')

In [20]:
df = prices.get(**kwargs_intraday, tzout="9988.HK")
df[:1]

symbol                                            MSFT                     \
                                                  open        high    low   
[2023-08-10 21:30:00, 2023-08-10 21:35:00)  326.015015  326.890015  325.5   

symbol                                                            9988.HK  \
                                                 close     volume    open   
[2023-08-10 21:30:00, 2023-08-10 21:35:00)  326.189911  1094887.0     NaN   

symbol                                                              AZN.L  \
                                           high low close volume     open   
[2023-08-10 21:30:00, 2023-08-10 21:35:00)  NaN NaN   NaN    NaN  11118.0   

symbol                                                                         
                                               high      low    close  volume  
[2023-08-10 21:30:00, 2023-08-10 21:35:00)  11132.0  11114.0  11120.0  8815.0

In [21]:
df.pt.tz

zoneinfo.ZoneInfo(key='Asia/Hong_Kong')

Finally, in the event a composite calendar is returned comprising daily and intraday data then the output will always be forced to UTC.

In [23]:
df_comp = prices.get(start=start_H1_oob, end=end, tzin="9988.HK", composite=True)
df_comp

symbol                                            MSFT              \
                                                  open        high   
[2021-09-08, 2021-09-08)                    299.779999  300.609985   
[2021-09-09, 2021-09-09)                    300.820007  302.140015   
[2021-09-10, 2021-09-10)                    298.420013  299.920013   
[2021-09-13, 2021-09-13)                    297.549988  298.540009   
[2021-09-14, 2021-09-14)                    299.559998  301.390015   
...                                                ...         ...   
[2023-08-10 19:35:00, 2023-08-10 19:40:00)  322.540009  322.940002   
[2023-08-10 19:40:00, 2023-08-10 19:45:00)  322.684998  322.984192   
[2023-08-10 19:45:00, 2023-08-10 19:50:00)  322.519989  323.269989   
[2023-08-10 19:50:00, 2023-08-10 19:55:00)  322.660004  323.383789   
[2023-08-10 19:55:00, 2023-08-10 20:00:00)  323.040009  323.299988   

symbol                                                              \
                                                   low       close   
[2021-09-08, 2021-09-08)                    297.470001  300.209991   
[2021-09-09, 2021-09-09)                    297.000000  297.250000   
[2021-09-10, 2021-09-10)                    295.380005  295.709991   
[2021-09-13, 2021-09-13)                    294.079987  296.989990   
[2021-09-14, 2021-09-14)                    298.100006  299.790009   
...                                                ...         ...   
[2023-08-10 19:35:00, 2023-08-10 19:40:00)  322.480011  322.670013   
[2023-08-10 19:40:00, 2023-08-10 19:45:00)  322.450012  322.540009   
[2023-08-10 19:45:00, 2023-08-10 19:50:00)  322.329987  322.690002   
[2023-08-10 19:50:00, 2023-08-10 19:55:00)  322.549988  323.089996   
[2023-08-10 19:55:00, 2023-08-10 20:00:00)  322.820007  322.950012   

symbol                                                     9988.HK  \
                                                volume        open   
[2021-09-08, 2021-09-08)                    15046800.0  170.000000   
[2021-09-09, 2021-09-09)                    19927000.0  165.000000   
[2021-09-10, 2021-09-10)                    19633400.0  163.699997   
[2021-09-13, 2021-09-13)                    23652900.0  161.500000   
[2021-09-14, 2021-09-14)                    21853400.0  160.399994   
...                                                ...         ...   
[2023-08-10 19:35:00, 2023-08-10 19:40:00)    125398.0         NaN   
[2023-08-10 19:40:00, 2023-08-10 19:45:00)    163060.0         NaN   
[2023-08-10 19:45:00, 2023-08-10 19:50:00)    170099.0         NaN   
[2023-08-10 19:50:00, 2023-08-10 19:55:00)    192185.0         NaN   
[2023-08-10 19:55:00, 2023-08-10 20:00:00)    747402.0         NaN   

symbol                                                              \
                                                  high         low   
[2021-09-08, 2021-09-08)                    173.399994  169.300003   
[2021-09-09, 2021-09-09)                    166.100006  160.300003   
[2021-09-10, 2021-09-10)                    168.199997  163.399994   
[2021-09-13, 2021-09-13)                    163.000000  156.899994   
[2021-09-14, 2021-09-14)                    162.100006  156.399994   
...                                                ...         ...   
[2023-08-10 19:35:00, 2023-08-10 19:40:00)         NaN         NaN   
[2023-08-10 19:40:00, 2023-08-10 19:45:00)         NaN         NaN   
[2023-08-10 19:45:00, 2023-08-10 19:50:00)         NaN         NaN   
[2023-08-10 19:50:00, 2023-08-10 19:55:00)         NaN         NaN   
[2023-08-10 19:55:00, 2023-08-10 20:00:00)         NaN         NaN   

symbol                                                               AZN.L  \
                                                 close      volume    open   
[2021-09-08, 2021-09-08)                    170.899994  24995877.0  8471.0   
[2021-09-09, 2021-09-09)                    161.000000  28379942.0  8300.0   
[2021-09-10, 2021-09-10)                    

In [24]:
df_comp.pt.tz

zoneinfo.ZoneInfo(key='UTC')

### `fill`

For each symbol `get` will always return prices for every indice that the calendar associated with the symbol is open. Missing values will only appear in indices during which the calendar associated with the symbol was closed. It follows that it's only possible for a prices table to have misssing values if symbols are associated with different calendars with different session times, as is the case in this example...

In [25]:
df_intraday

symbol                                            MSFT              \
                                                  open        high   
[2023-08-10 09:30:00, 2023-08-10 09:35:00)  326.015015  326.890015   
[2023-08-10 09:35:00, 2023-08-10 09:40:00)  326.149994  326.636505   
[2023-08-10 09:40:00, 2023-08-10 09:45:00)  326.200012  326.570007   
[2023-08-10 09:45:00, 2023-08-10 09:50:00)  325.450012  326.510010   
[2023-08-10 09:50:00, 2023-08-10 09:55:00)  326.399994  326.579987   
...                                                ...         ...   
[2023-08-11 15:35:00, 2023-08-11 15:40:00)  321.720001  321.829987   
[2023-08-11 15:40:00, 2023-08-11 15:45:00)  321.339996  321.429993   
[2023-08-11 15:45:00, 2023-08-11 15:50:00)  321.350006  321.399994   
[2023-08-11 15:50:00, 2023-08-11 15:55:00)  321.190002  321.440002   
[2023-08-11 15:55:00, 2023-08-11 16:00:00)  320.894989  321.269989   

symbol                                                                         \
                                                   low       close     volume   
[2023-08-10 09:30:00, 2023-08-10 09:35:00)  325.500000  326.189911  1094887.0   
[2023-08-10 09:35:00, 2023-08-10 09:40:00)  325.489990  326.160004   365134.0   
[2023-08-10 09:40:00, 2023-08-10 09:45:00)  325.109985  325.340088   345965.0   
[2023-08-10 09:45:00, 2023-08-10 09:50:00)  325.450012  326.390015   304905.0   
[2023-08-10 09:50:00, 2023-08-10 09:55:00)  325.950012  326.440002   325788.0   
...                                                ...         ...        ...   
[2023-08-11 15:35:00, 2023-08-11 15:40:00)  321.279999  321.353210   165967.0   
[2023-08-11 15:40:00, 2023-08-11 15:45:00)  321.179993  321.338013   217661.0   
[2023-08-11 15:45:00, 2023-08-11 15:50:00)  320.910004  321.179993   241240.0   
[2023-08-11 15:50:00, 2023-08-11 15:55:00)  320.859985  320.910004   389375.0   
[2023-08-11 15:55:00, 2023-08-11 16:00:00)  320.589996  321.089996  1342284.0   

symbol                                     9988.HK                        \
                                              open high low close volume   
[2023-08-10 09:30:00, 2023-08-10 09:35:00)     NaN  NaN NaN   NaN    NaN   
[2023-08-10 09:35:00, 2023-08-10 09:40:00)     NaN  NaN NaN   NaN    NaN   
[2023-08-10 09:40:00, 2023-08-10 09:45:00)     NaN  NaN NaN   NaN    NaN   
[2023-08-10 09:45:00, 2023-08-10 09:50:00)     NaN  NaN NaN   NaN    NaN   
[2023-08-10 09:50:00, 2023-08-10 09:55:00)     NaN  NaN NaN   NaN    NaN   
...                                            ...  ...  ..   ...    ...   
[2023-08-11 15:35:00, 2023-08-11 15:40:00)     NaN  NaN NaN   NaN    NaN   
[2023-08-11 15:40:00, 2023-08-11 15:45:00)     NaN  NaN NaN   NaN    NaN   
[2023-08-11 15:45:00, 2023-08-11 15:50:00)     NaN  NaN NaN   NaN    NaN   
[2023-08-11 15:50:00, 2023-08-11 15:55:00)     NaN  NaN NaN   NaN    NaN   
[2023-08-11 15:55:00, 2023-08-11 16:00:00)     NaN  NaN NaN   NaN    NaN   

symbol                                        AZN.L                    \
                                               open     high      low   
[2023-08-10 09:30:00, 2023-08-10 09:35:00)  11118.0  11132.0  11114.0   
[2023-08-10 09:35:00, 2023-08-10 09:40:00)  11122.0  11134.0  11112.0   
[2023-08-10 09:40:00, 2023-08-10 09:45:00)  11114.0  11138.0  11114.0   
[2023-08-10 09:45:00, 2023-08-10 09:50:00)  11124.0  11138.0  11118.0   
[2023-08-10 09:50:00, 2023-08-10 09:55:00)  11120.0  11140.0  11116.0   
...                                             ...      ...      ...   
[2023-08-11 15:35:00, 2023-08-11 15:40:00)      NaN      NaN      NaN   
[2023-08-11 15:40:00, 2023-08-11 15:45:00)      NaN      NaN      NaN   
[2023-08-11 15:45:00, 2023-08-11 15:50:00)      NaN      NaN      NaN   
[2023-08-11 15:50:00, 2023-08-11 15:55:00)      NaN      NaN      NaN   
[2023-08-11 15:55:00, 2023-08-11 16:00:00)      NaN      NaN      NaN   

symbol                                                        
                                 

The `fill` parameter exists to optionally fill those missing values when a symbol's exchange was closed. The parameter can take the following values:
* **"ffill"** to fill 'open', 'close', 'high' and 'low' values with the last available close price.
* **"bfill"** to fill 'open', 'close', 'high' and 'low' values with the next available open price.
* **"both"** to first fill as "ffill" and then fill any missing values at the start of the table with "bfill".

The `volume` column will always be filled with 0.

In [26]:
prices.get(**kwargs_intraday, fill="ffill")

symbol                                            MSFT              \
                                                  open        high   
[2023-08-10 09:30:00, 2023-08-10 09:35:00)  326.015015  326.890015   
[2023-08-10 09:35:00, 2023-08-10 09:40:00)  326.149994  326.636505   
[2023-08-10 09:40:00, 2023-08-10 09:45:00)  326.200012  326.570007   
[2023-08-10 09:45:00, 2023-08-10 09:50:00)  325.450012  326.510010   
[2023-08-10 09:50:00, 2023-08-10 09:55:00)  326.399994  326.579987   
...                                                ...         ...   
[2023-08-11 15:35:00, 2023-08-11 15:40:00)  321.720001  321.829987   
[2023-08-11 15:40:00, 2023-08-11 15:45:00)  321.339996  321.429993   
[2023-08-11 15:45:00, 2023-08-11 15:50:00)  321.350006  321.399994   
[2023-08-11 15:50:00, 2023-08-11 15:55:00)  321.190002  321.440002   
[2023-08-11 15:55:00, 2023-08-11 16:00:00)  320.894989  321.269989   

symbol                                                                         \
                                                   low       close     volume   
[2023-08-10 09:30:00, 2023-08-10 09:35:00)  325.500000  326.189911  1094887.0   
[2023-08-10 09:35:00, 2023-08-10 09:40:00)  325.489990  326.160004   365134.0   
[2023-08-10 09:40:00, 2023-08-10 09:45:00)  325.109985  325.340088   345965.0   
[2023-08-10 09:45:00, 2023-08-10 09:50:00)  325.450012  326.390015   304905.0   
[2023-08-10 09:50:00, 2023-08-10 09:55:00)  325.950012  326.440002   325788.0   
...                                                ...         ...        ...   
[2023-08-11 15:35:00, 2023-08-11 15:40:00)  321.279999  321.353210   165967.0   
[2023-08-11 15:40:00, 2023-08-11 15:45:00)  321.179993  321.338013   217661.0   
[2023-08-11 15:45:00, 2023-08-11 15:50:00)  320.910004  321.179993   241240.0   
[2023-08-11 15:50:00, 2023-08-11 15:55:00)  320.859985  320.910004   389375.0   
[2023-08-11 15:55:00, 2023-08-11 16:00:00)  320.589996  321.089996  1342284.0   

symbol                                        9988.HK                        \
                                                 open       high        low   
[2023-08-10 09:30:00, 2023-08-10 09:35:00)        NaN        NaN        NaN   
[2023-08-10 09:35:00, 2023-08-10 09:40:00)        NaN        NaN        NaN   
[2023-08-10 09:40:00, 2023-08-10 09:45:00)        NaN        NaN        NaN   
[2023-08-10 09:45:00, 2023-08-10 09:50:00)        NaN        NaN        NaN   
[2023-08-10 09:50:00, 2023-08-10 09:55:00)        NaN        NaN        NaN   
...                                               ...        ...        ...   
[2023-08-11 15:35:00, 2023-08-11 15:40:00)  95.300003  95.300003  95.300003   
[2023-08-11 15:40:00, 2023-08-11 15:45:00)  95.300003  95.300003  95.300003   
[2023-08-11 15:45:00, 2023-08-11 15:50:00)  95.300003  95.300003  95.300003   
[2023-08-11 15:50:00, 2023-08-11 15:55:00)  95.300003  95.300003  95.300003   
[2023-08-11 15:55:00, 2023-08-11 16:00:00)  95.300003  95.300003  95.300003   

symbol                                                          AZN.L  \
                                                close volume     open   
[2023-08-10 09:30:00, 2023-08-10 09:35:00)        NaN    NaN  11118.0   
[2023-08-10 09:35:00, 2023-08-10 09:40:00)        NaN    NaN  11122.0   
[2023-08-10 09:40:00, 2023-08-10 09:45:00)        NaN    NaN  11114.0   
[2023-08-10 09:45:00, 2023-08-10 09:50:00)        NaN    NaN  11124.0   
[2023-08-10 09:50:00, 2023-08-10 09:55:00)        NaN    NaN  11120.0   
...                                               ...    ...      ...   
[2023-08-11 15:35:00, 2023-08-11 15:40:00)  95.300003    0.0  11022.0   
[2023-08-11 15:40:00, 2023-08-11 15:45:00)  95.300003    0.0  11022.0   
[2023-08-11 15:45:00, 2023-08-11 15:50:00)  95.300003    0.0  11022.0   
[2023-08-11 15:50:00, 2023-08-11 15:55:00)  95.300003    0.0  11022.0   
[2023-08-11 15:55:00, 2023-08-11 16:00:00)  95.300003    0.0  11022.0   

symbol                                                   

In [27]:
prices.get(**kwargs_intraday, fill="both")

symbol                                            MSFT              \
                                                  open        high   
[2023-08-10 09:30:00, 2023-08-10 09:35:00)  326.015015  326.890015   
[2023-08-10 09:35:00, 2023-08-10 09:40:00)  326.149994  326.636505   
[2023-08-10 09:40:00, 2023-08-10 09:45:00)  326.200012  326.570007   
[2023-08-10 09:45:00, 2023-08-10 09:50:00)  325.450012  326.510010   
[2023-08-10 09:50:00, 2023-08-10 09:55:00)  326.399994  326.579987   
...                                                ...         ...   
[2023-08-11 15:35:00, 2023-08-11 15:40:00)  321.720001  321.829987   
[2023-08-11 15:40:00, 2023-08-11 15:45:00)  321.339996  321.429993   
[2023-08-11 15:45:00, 2023-08-11 15:50:00)  321.350006  321.399994   
[2023-08-11 15:50:00, 2023-08-11 15:55:00)  321.190002  321.440002   
[2023-08-11 15:55:00, 2023-08-11 16:00:00)  320.894989  321.269989   

symbol                                                                         \
                                                   low       close     volume   
[2023-08-10 09:30:00, 2023-08-10 09:35:00)  325.500000  326.189911  1094887.0   
[2023-08-10 09:35:00, 2023-08-10 09:40:00)  325.489990  326.160004   365134.0   
[2023-08-10 09:40:00, 2023-08-10 09:45:00)  325.109985  325.340088   345965.0   
[2023-08-10 09:45:00, 2023-08-10 09:50:00)  325.450012  326.390015   304905.0   
[2023-08-10 09:50:00, 2023-08-10 09:55:00)  325.950012  326.440002   325788.0   
...                                                ...         ...        ...   
[2023-08-11 15:35:00, 2023-08-11 15:40:00)  321.279999  321.353210   165967.0   
[2023-08-11 15:40:00, 2023-08-11 15:45:00)  321.179993  321.338013   217661.0   
[2023-08-11 15:45:00, 2023-08-11 15:50:00)  320.910004  321.179993   241240.0   
[2023-08-11 15:50:00, 2023-08-11 15:55:00)  320.859985  320.910004   389375.0   
[2023-08-11 15:55:00, 2023-08-11 16:00:00)  320.589996  321.089996  1342284.0   

symbol                                        9988.HK                        \
                                                 open       high        low   
[2023-08-10 09:30:00, 2023-08-10 09:35:00)  97.099998  97.099998  97.099998   
[2023-08-10 09:35:00, 2023-08-10 09:40:00)  97.099998  97.099998  97.099998   
[2023-08-10 09:40:00, 2023-08-10 09:45:00)  97.099998  97.099998  97.099998   
[2023-08-10 09:45:00, 2023-08-10 09:50:00)  97.099998  97.099998  97.099998   
[2023-08-10 09:50:00, 2023-08-10 09:55:00)  97.099998  97.099998  97.099998   
...                                               ...        ...        ...   
[2023-08-11 15:35:00, 2023-08-11 15:40:00)  95.300003  95.300003  95.300003   
[2023-08-11 15:40:00, 2023-08-11 15:45:00)  95.300003  95.300003  95.300003   
[2023-08-11 15:45:00, 2023-08-11 15:50:00)  95.300003  95.300003  95.300003   
[2023-08-11 15:50:00, 2023-08-11 15:55:00)  95.300003  95.300003  95.300003   
[2023-08-11 15:55:00, 2023-08-11 16:00:00)  95.300003  95.300003  95.300003   

symbol                                                          AZN.L  \
                                                close volume     open   
[2023-08-10 09:30:00, 2023-08-10 09:35:00)  97.099998    0.0  11118.0   
[2023-08-10 09:35:00, 2023-08-10 09:40:00)  97.099998    0.0  11122.0   
[2023-08-10 09:40:00, 2023-08-10 09:45:00)  97.099998    0.0  11114.0   
[2023-08-10 09:45:00, 2023-08-10 09:50:00)  97.099998    0.0  11124.0   
[2023-08-10 09:50:00, 2023-08-10 09:55:00)  97.099998    0.0  11120.0   
...                                               ...    ...      ...   
[2023-08-11 15:35:00, 2023-08-11 15:40:00)  95.300003    0.0  11022.0   
[2023-08-11 15:40:00, 2023-08-11 15:45:00)  95.300003    0.0  11022.0   
[2023-08-11 15:45:00, 2023-08-11 15:50:00)  95.300003    0.0  11022.0   
[2023-08-11 15:50:00, 2023-08-11 15:55:00)  95.300003    0.0  11022.0   
[2023-08-11 15:55:00, 2023-08-11 16:00:00)  95.300003    0.0  11022.0   

symbol                                                   

With daily data `fill` will fill values for sessions when an exchange was not open. Compare the following...

In [28]:
prices_alt = PricesYahoo("MSFT, AZN.L, 9988.HK", lead_symbol="AZN.L")
args = ("1D", "2021-12-24", "2021-12-29")
prices_alt.get(*args)

symbol            MSFT                                                  \
                  open        high         low       close      volume   
2021-12-24         NaN         NaN         NaN         NaN         NaN   
2021-12-27  335.459991  342.480011  335.429993  342.450012  19947000.0   
2021-12-28  343.149994  343.809998  340.320007  341.250000  15661500.0   
2021-12-29  341.299988  344.299988  339.679993  341.950012  15042000.0   

symbol       AZN.L                                             9988.HK  \
              open    high          low   close     volume        open   
2021-12-24  8613.0  8676.0  8608.487305  8611.0   234238.0  113.000000   
2021-12-27     NaN     NaN          NaN     NaN        NaN         NaN   
2021-12-28     NaN     NaN          NaN     NaN        NaN  113.300003   
2021-12-29  8657.0  8720.0  8651.545898  8656.0  1406164.0  112.300003   

symbol                                                      
                  high         low       close      volume  
2021-12-24  113.000000  113.000000  113.000000         0.0  
2021-12-27         NaN         NaN         NaN         NaN  
2021-12-28  114.199997  112.300003  113.500000  22985751.0  
2021-12-29  112.300003  110.099998  110.599998  19990686.0

In [29]:
prices_alt.get(*args, fill="both")

symbol            MSFT                                                  \
                  open        high         low       close      volume   
2021-12-24  335.459991  335.459991  335.459991  335.459991         0.0   
2021-12-27  335.459991  342.480011  335.429993  342.450012  19947000.0   
2021-12-28  343.149994  343.809998  340.320007  341.250000  15661500.0   
2021-12-29  341.299988  344.299988  339.679993  341.950012  15042000.0   

symbol       AZN.L                                             9988.HK  \
              open    high          low   close     volume        open   
2021-12-24  8613.0  8676.0  8608.487305  8611.0   234238.0  113.000000   
2021-12-27  8611.0  8611.0  8611.000000  8611.0        0.0  113.000000   
2021-12-28  8611.0  8611.0  8611.000000  8611.0        0.0  113.300003   
2021-12-29  8657.0  8720.0  8651.545898  8656.0  1406164.0  112.300003   

symbol                                                      
                  high         low       close      volume  
2021-12-24  113.000000  113.000000  113.000000         0.0  
2021-12-27  113.000000  113.000000  113.000000         0.0  
2021-12-28  114.199997  112.300003  113.500000  22985751.0  
2021-12-29  112.300003  110.099998  110.599998  19990686.0

Notice how the missing prices for MSFT on the 24th are filled backwards with the open price on the 27th, whilst the missing prices for AZN.L are filled fowards with the close price on the 24th.

To maximise the possibility of filling even the initial row forwards, you can always see if the prior indice had a value that could be carried forward... 

In [30]:
df = prices_alt.get(*args, add_a_row=True, fill="ffill")
df[1:]

symbol            MSFT                                                  \
                  open        high         low       close      volume   
2021-12-24  334.690002  334.690002  334.690002  334.690002         0.0   
2021-12-27  335.459991  342.480011  335.429993  342.450012  19947000.0   
2021-12-28  343.149994  343.809998  340.320007  341.250000  15661500.0   
2021-12-29  341.299988  344.299988  339.679993  341.950012  15042000.0   

symbol       AZN.L                                             9988.HK  \
              open    high          low   close     volume        open   
2021-12-24  8613.0  8676.0  8608.487305  8611.0   234238.0  113.000000   
2021-12-27  8611.0  8611.0  8611.000000  8611.0        0.0  113.000000   
2021-12-28  8611.0  8611.0  8611.000000  8611.0        0.0  113.300003   
2021-12-29  8657.0  8720.0  8651.545898  8656.0  1406164.0  112.300003   

symbol                                                      
                  high         low       close      volume  
2021-12-24  113.000000  113.000000  113.000000         0.0  
2021-12-27  113.000000  113.000000  113.000000         0.0  
2021-12-28  114.199997  112.300003  113.500000  22985751.0  
2021-12-29  112.300003  110.099998  110.599998  19990686.0

### `include` and `exclude`

* `include` takes a symbol or list of symbols to include in the returned table.
* `exclude` takes a symbol or list of symbols to exclude from the returned table.

Only one or neither of `include` and `exclude` can be passed.

In [31]:
prices.get(**kwargs_intraday, include="MSFT")[:2]

symbol                                            MSFT                         \
                                                  open        high        low   
[2023-08-10 09:30:00, 2023-08-10 09:35:00)  326.015015  326.890015  325.50000   
[2023-08-10 09:35:00, 2023-08-10 09:40:00)  326.149994  326.636505  325.48999   

symbol                                                             
                                                 close     volume  
[2023-08-10 09:30:00, 2023-08-10 09:35:00)  326.189911  1094887.0  
[2023-08-10 09:35:00, 2023-08-10 09:40:00)  326.160004   365134.0

In [32]:
prices.get(**kwargs_daily, exclude=["MSFT", "AZN.L"])[:2]

symbol        9988.HK                                          
                 open    high        low      close      volume
2023-07-31  99.000000   99.75  96.800003  97.500000  74290881.0
2023-08-01  99.699997  100.00  96.900002  97.849998  53949339.0

**NOTE**: When passing `include` or `exclude` it's important to recall that they are applied as a post-processing operation after the table has been created. Excluding a symbol will consequently have no affect on, for example, the evaluation of a final indice where the period end evalutes as an unaligned session close, which will still be evaluated as if prices included all symbols (see [openend](./anchor.ipynb#openend) section of [anchor](./anchor.ipynb) tutorial for explanation of how such final indices are evaluated).

### `side`

`side` can be passed to index a price table with a `pd.DateTimeIndex` when it would otherwise be indexed with a `pd.IntervalIndex`.

`side` can optionally take either "left" or "right" to define each indice as the, respectively, left or right side of the period that the indice represents.

In [33]:
df_intraday[:2]  # for comparison

symbol                                            MSFT                         \
                                                  open        high        low   
[2023-08-10 09:30:00, 2023-08-10 09:35:00)  326.015015  326.890015  325.50000   
[2023-08-10 09:35:00, 2023-08-10 09:40:00)  326.149994  326.636505  325.48999   

symbol                                                            9988.HK  \
                                                 close     volume    open   
[2023-08-10 09:30:00, 2023-08-10 09:35:00)  326.189911  1094887.0     NaN   
[2023-08-10 09:35:00, 2023-08-10 09:40:00)  326.160004   365134.0     NaN   

symbol                                                              AZN.L  \
                                           high low close volume     open   
[2023-08-10 09:30:00, 2023-08-10 09:35:00)  NaN NaN   NaN    NaN  11118.0   
[2023-08-10 09:35:00, 2023-08-10 09:40:00)  NaN NaN   NaN    NaN  11122.0   

symbol                                                                          
                                               high      low    close   volume  
[2023-08-10 09:30:00, 2023-08-10 09:35:00)  11132.0  11114.0  11120.0   8815.0  
[2023-08-10 09:35:00, 2023-08-10 09:40:00)  11134.0  11112.0  11112.0  12659.0

In [34]:
prices.get(**kwargs_intraday, side="left")[:2]

symbol                           MSFT                                     \
                                 open        high        low       close   
2023-08-10 09:30:00-04:00  326.015015  326.890015  325.50000  326.189911   
2023-08-10 09:35:00-04:00  326.149994  326.636505  325.48999  326.160004   

symbol                               9988.HK                          AZN.L  \
                              volume    open high low close volume     open   
2023-08-10 09:30:00-04:00  1094887.0     NaN  NaN NaN   NaN    NaN  11118.0   
2023-08-10 09:35:00-04:00   365134.0     NaN  NaN NaN   NaN    NaN  11122.0   

symbol                                                         
                              high      low    close   volume  
2023-08-10 09:30:00-04:00  11132.0  11114.0  11120.0   8815.0  
2023-08-10 09:35:00-04:00  11134.0  11112.0  11112.0  12659.0

In [35]:
df = prices.get(**kwargs_intraday, side="right")
df[:2]

symbol                           MSFT                                     \
                                 open        high        low       close   
2023-08-10 09:35:00-04:00  326.015015  326.890015  325.50000  326.189911   
2023-08-10 09:40:00-04:00  326.149994  326.636505  325.48999  326.160004   

symbol                               9988.HK                          AZN.L  \
                              volume    open high low close volume     open   
2023-08-10 09:35:00-04:00  1094887.0     NaN  NaN NaN   NaN    NaN  11118.0   
2023-08-10 09:40:00-04:00   365134.0     NaN  NaN NaN   NaN    NaN  11122.0   

symbol                                                         
                              high      low    close   volume  
2023-08-10 09:35:00-04:00  11132.0  11114.0  11120.0   8815.0  
2023-08-10 09:40:00-04:00  11134.0  11112.0  11112.0  12659.0

NOTE: If side is passed then the returned prices table will not have access to the `.pt` accessor.

In [ ]:
df.pt

```
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
c:\market_prices\docs\tutorials\other_get_options.ipynb Cell 62 line 1
----> 1 df.pt

ValueError: PT accessor not available where index is a pd.DatatimeIndex with one or more indices that have a time component (Index must be pd.IntervalIndex if any indice includes a time component).
```

### `close_only`

`close_only` can be passed as `True` to return only the close price columns for each symbol.

In [37]:
prices.get(**kwargs_daily, close_only=True)[:4]

symbol,MSFT,9988.HK,AZN.L
2023-07-31,335.920013,97.500000,11186.0
2023-08-01,336.339996,97.849998,11196.0
2023-08-02,327.500000,95.150002,11096.0
2023-08-03,326.660004,93.150002,10900.0


NOTE: if `close_only` is passed as True then the returned price table will not have access to the `.pt` accessor.

### `lose_single_symbol`

**`lose_single_symbol`** is only relevant if the table comprises a single symbol, in which case can be used to remove the symbol level of the columns index.

In [38]:
prices_us = PricesYahoo("MSFT")

In [39]:
prices_us.get(**kwargs_intraday)[:3]

symbol                                            MSFT              \
                                                  open        high   
[2023-08-10 09:30:00, 2023-08-10 10:30:00)  326.015015  328.260010   
[2023-08-10 10:30:00, 2023-08-10 11:30:00)  326.230011  327.174988   
[2023-08-10 11:30:00, 2023-08-10 12:30:00)  324.260010  325.049988   

symbol                                                                       
                                                   low       close   volume  
[2023-08-10 09:30:00, 2023-08-10 10:30:00)  325.109985  326.250000  4302172  
[2023-08-10 10:30:00, 2023-08-10 11:30:00)  323.040009  324.234985  2950799  
[2023-08-10 11:30:00, 2023-08-10 12:30:00)  322.079987  323.250000  2184202

In [40]:
prices_us.get(**kwargs_intraday, lose_single_symbol=True)[:3]

,open,high,low,close,volume
"[2023-08-10 09:30:00, 2023-08-10 10:30:00)",326.015015,328.260010,325.109985,326.250000,4302172
"[2023-08-10 10:30:00, 2023-08-10 11:30:00)",326.230011,327.174988,323.040009,324.234985,2950799
"[2023-08-10 11:30:00, 2023-08-10 12:30:00)",324.260010,325.049988,322.079987,323.250000,2184202


NOTE: `lose_single_symbol` is ignored if `close_only` is passed as `True`.